In [ ]:
import joblib
import numpy as np
import os
import tensorflow as tf
import math
import csv

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, CSVLogger
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
from keras.utils import plot_model
from PIL import Image
from skimage.metrics import structural_similarity as ssim
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt # to plot images
from google.colab import drive

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-ca2b66e117e2>", line 16, in <cell line: 16>
    from skimage.metrics import structural_similarity as ssim
  File "/usr/local/lib/python3.10/dist-packages/skimage/metrics/__init__.py", line 4, in <module>
    from .simple_metrics import (mean_squared_error,
  File "/usr/local/lib/python3.10/dist-packages/skimage/metrics/simple_metrics.py", line 2, in <module>
    from scipy.stats import entropy
  File "/usr/local/lib/python3.10/dist-packages/scipy/stats/__init__.py", line 485, in <module>
    from ._stats_py import *
  File "/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py", line 46, in <module>
    from . import distributions
  File "/usr/local/lib/python3.10/dist-packages/scipy/stats/distributions.py", line 8, in <module>
    from ._distn_infrastructure

TypeError: ignored

In [ ]:
drive.mount("/content/drive")

In [ ]:
with open("/content/drive/MyDrive/ColabFiles/uNet-cc/train-and-val.pkl", "rb") as file:
  train_and_val_dataset_file = joblib.load(file)
with open("/content/drive/MyDrive/ColabFiles/uNet-cc/test.pkl", "rb") as file:
  test_dataset_file = joblib.load(file)

In [ ]:
X_remaining, Y_remaining, remaining_dataset_desc = train_and_val_dataset_file
Xte, yte, test_dataset_desc = test_dataset_file
training_set_index = remaining_dataset_desc['training_set_index']
validation_set_index = remaining_dataset_desc['validation_set_index']
saved_model_filename = "datasets/dataset-1/test-4-new-tentative-{epoch:02d}-{val_dice_coef_accur:.4f}.hdf5"
Xtr, ytr = X_remaining[:training_set_index,:], Y_remaining[:training_set_index]
Xva, yva = X_remaining[training_set_index:validation_set_index,:], Y_remaining[training_set_index:validation_set_index]

In [ ]:
#(batch_size, height, width, channels)
K.set_image_data_format("channels_last")
_, *input_image_shape, _ = Xtr.shape
input_image_shape = tuple(input_image_shape)
print(input_image_shape)

smooth = 1

use_dropout = True
use_regularizers = True
dropout_rate = 0.5
number_of_epochs = 1000
batch_size = 32
kernel_size = (5, 5)
initial_volume_size = 64

(128, 128)


In [ ]:
# Dice loss coefficient
# Print y_true and y_pred before doing function!
def dice_coef(y_true, y_pred):
  y_true_f = K.flatten(y_true)
  y_pred_f = K.flatten(y_pred)
  intersection = 2 * K.sum(y_true_f * y_pred_f) + smooth
  union = K.sum(y_true_f) + K.sum(y_pred_f) + smooth
  return K.mean(intersection / union)

def dice_coef_per_image_in_batch(y_true, y_pred):
  y_true_f = K.batch_flatten(y_true)
  y_pred_f = K.batch_flatten(y_pred)
  intersection = 2. * K.sum(y_true_f * y_pred_f, axis=1, keepdims=True) + smooth
  union = K.sum(y_true_f, axis=1, keepdims=True) + K.sum(y_pred_f, axis=1, keepdims=True) + smooth
  return K.mean(intersection / union)

def dice_coef_loss(y_true, y_pred):
  return -dice_coef_per_image_in_batch(y_true, y_pred)

def dice_coef_accur(y_true, y_pred):
  return dice_coef_per_image_in_batch(y_true, y_pred)

def IOU_calc(y_true, y_pred):
  y_true_f = K.flatten(y_true)
  y_pred_f = K.flatten(y_pred)
  intersection = K.sum(y_true_f * y_pred_f)

  return 2*(intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def IOU_calc_loss(y_true, y_pred):
  return -IOU_calc(y_true, y_pred)

def setup_regularizers(conv_layer):
  return BatchNormalization()(conv_layer) if use_regularizers else conv_layer

def setup_dropout(conv_layer):
  return Dropout(dropout_rate)(conv_layer) if use_dropout else conv_layer


In [ ]:
inputs = Input((*input_image_shape, 1))
conv1 = Conv2D(initial_volume_size, kernel_size, activation = "relu", padding = "same")(inputs)
conv1 = Conv2D(initial_volume_size, kernel_size, activation = "relu", padding = "same")(conv1)
conv1 = setup_regularizers(conv1)
pool1 = MaxPooling2D(pool_size = (2, 2))(conv1)

conv2 = Conv2D(initial_volume_size * 2, kernel_size, activation = "relu", padding = "same")(pool1)
conv2 = Conv2D(initial_volume_size * 2, kernel_size, activation = "relu", padding = "same")(conv2)
conv2 = setup_regularizers(conv2)
pool2 = MaxPooling2D(pool_size = (2, 2))(conv2)

conv3 = Conv2D(initial_volume_size * 4, kernel_size, activation = "relu", padding = "same")(pool2)
conv3 = Conv2D(initial_volume_size * 4, kernel_size, activation = "relu", padding = "same")(conv3)
conv3 = setup_regularizers(conv3)
pool3 = MaxPooling2D(pool_size = (2, 2))(conv3)

conv4 = Conv2D(initial_volume_size * 8, kernel_size, activation = "relu", padding = "same")(pool3)
conv4 = Conv2D(initial_volume_size * 8, kernel_size, activation = "relu", padding = "same")(conv4)
conv4 = setup_regularizers(conv4)
pool4 = MaxPooling2D(pool_size = (2, 2))(conv4)

conv5 = Conv2D(initial_volume_size * 16, kernel_size, activation = "relu", padding = 'same')(pool4)
conv5 = Conv2D(initial_volume_size * 16, kernel_size, activation = "relu", padding = "same")(conv5)
conv5 = setup_regularizers(conv5)
pool5 = MaxPooling2D(pool_size = (2, 2))(conv5)

up6 = concatenate([UpSampling2D(size = (2, 2))(conv5), conv4], axis = 3)
up6 = setup_dropout(up6)
conv6 = Conv2D(initial_volume_size * 8, kernel_size, activation = "relu", padding = "same")(up6)
conv6 = Conv2D(initial_volume_size * 8, kernel_size, activation = "relu", padding = "same")(conv6)

up7 = concatenate([UpSampling2D(size = (2, 2))(conv6), conv3], axis = 3)
up7 = setup_dropout(up7)

conv7 = Conv2D(initial_volume_size * 4, kernel_size, activation = "relu", padding = "same")(up7)
conv7 = Conv2D(initial_volume_size * 4, kernel_size, activation = "relu", padding = "same")(conv7)

up8 = concatenate([UpSampling2D(size = (2, 2))(conv7), conv2], axis = 3)
up8 = setup_dropout(up8)

conv8 = Conv2D(initial_volume_size * 2, kernel_size, activation = "relu", padding = "same")(up8)
conv8 = Conv2D(initial_volume_size * 2, kernel_size, activation = "relu", padding = "same")(conv8)

up9 = concatenate([UpSampling2D(size = (2, 2))(conv8), conv1], axis = 3)
up9 = setup_dropout(up9)

conv9 = Conv2D(initial_volume_size, kernel_size, activation = "relu", padding = "same")(up9)
conv9 = Conv2D(initial_volume_size, kernel_size, activation = "relu", padding = 'same')(conv9)

conv10 = Conv2D(1, (1, 1), activation = "sigmoid")(conv9)

model = Model(inputs = [inputs], outputs = [conv10])
model.compile(
    optimizer = Adam(lr = 1e-5),
    loss = dice_coef_loss,
    metrics = [dice_coef_accur]
)

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 64  1664        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 128, 128, 64  102464      ['conv2d[0][0]']                 
                                )                                                             

In [ ]:
model_checkpoint = ModelCheckpoint(saved_model_filename, monitor='val_dice_coef_accur', save_best_only=True, verbose=1)

history = model.fit(Xtr,
                    y=ytr,
                    batch_size = batch_size,
                    epochs = number_of_epochs,
                    verbose = 2,
                    shuffle = True,
                    callbacks = [model_checkpoint],
                    validation_data = (Xva, yva))

Epoch 1/1000

Epoch 1: val_dice_coef_accur improved from -inf to 0.20364, saving model to datasets/dataset-1/test-4-new-tentative-01-0.2036.hdf5
44/44 - 108s - loss: -2.9997e-01 - dice_coef_accur: 0.3016 - val_loss: -2.0013e-01 - val_dice_coef_accur: 0.2036 - 108s/epoch - 2s/step
Epoch 2/1000

Epoch 2: val_dice_coef_accur did not improve from 0.20364
44/44 - 37s - loss: -7.9272e-01 - dice_coef_accur: 0.7929 - val_loss: -3.2189e-02 - val_dice_coef_accur: 0.0292 - 37s/epoch - 850ms/step
Epoch 3/1000

Epoch 3: val_dice_coef_accur did not improve from 0.20364
44/44 - 37s - loss: -8.6742e-01 - dice_coef_accur: 0.8675 - val_loss: -1.2492e-02 - val_dice_coef_accur: 0.0116 - 37s/epoch - 848ms/step
Epoch 4/1000

Epoch 4: val_dice_coef_accur did not improve from 0.20364
44/44 - 37s - loss: -8.9155e-01 - dice_coef_accur: 0.8916 - val_loss: -7.0272e-03 - val_dice_coef_accur: 0.0067 - 37s/epoch - 848ms/step
Epoch 5/1000

Epoch 5: val_dice_coef_accur did not improve from 0.20364
44/44 - 36s - loss: 